In [8]:
#pip install python-steam-api
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import requests
import time
import pandas as pd
import numpy as np
import json
import pickleshare

The following chunk runs a selenium browser to open up steams most played games and create a list of links to those game pages. 
Then it extracts the app_id of the game and appends it to our api link to create a list of api's with the app ids we are interested in. 

In [9]:
user_agent = "Microsoft Edge UA string:Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10136"
options = webdriver.EdgeOptions()
options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument("--headless=new")

driver = webdriver.Edge()

url = "https://store.steampowered.com/charts/mostplayed/"

links = []
app_ids = []

def get_app_ids(url, rec=False):
    page = driver.get(url)
    response = requests.get(url, user_agent)
    print(response.status_code, url)


    time.sleep(2)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN) 

        
    soup = BeautifulSoup(driver.page_source, "html")

    chart = soup.find("div", class_ = "_1A7NagdRz58_o8HPHMa3eE").find("table", class_ = "_3arZn0BMPzyhcYNADe193m")

    for ls in chart.find_all("td"):
        try:  
            link = (ls.find("a", href = True)["href"])
            links.append(link)
            app_id = link.split('/')[4]
            app_ids.append(app_id)

        except Exception as e:   
            None

get_app_ids(url)
    
print(app_ids)


driver.quit()

200 https://store.steampowered.com/charts/mostplayed/
['730', '570', '578080', '2246340', '3241660', '252490', '2767030', '359550', '271590', '2669320', '236390', '1172470', '3240220', '1086940', '221100', '2252570', '2943650', '1771300', '431960', '1366800', '413150', '1938090', '440', '230410', '394360', '227300', '2507950', '1245620', '2923300', '1222670', '381210', '2429640', '1174180', '1782210', '1085660', '553850', '892970', '289070', '1091500', '629520', '2694490', '438100', '2001120', '105600', '2300320', '1281930', '489830', '1623730', '108600', '346110', '252950', '251570', '3205720', '3419430', '250900', '2399830', '4000', '1142710', '1158310', '294100', '2379780', '2357570', '739630', '526870', '550', '284160', '238960', '1203220', '427520', '1326470', '322170', '322330', '813780', '2139460', '393380', '582660', '379430', '1295660', '1238840', '236850', '39210', '513710', '261550', '582010', '2878980', '292030', '8930', '1449850', '377160', '2074920', '10', '240', '990080'

In [10]:
%store app_ids

Stored 'app_ids' (list)


In [11]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey, Date, URL
from typing import List
from typing import Optional
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import create_engine
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker

db_url = URL.create(
    drivername='postgresql+psycopg2', 
    username = 'postgres', 
    password='CbassEngineering1003.', 
    host='localhost', 
    port="5432", 
    database='steam_games_details'
)

engine = create_engine(db_url, pool_pre_ping=True)

Base = DeclarativeBase

class Base(DeclarativeBase): 
    pass

class steamGames(Base): 
    __tablename__ = 'games'
    gameid = mapped_column(Integer, primary_key=True)
    name = mapped_column(String)
    description = mapped_column(String)
    price = mapped_column(String)
    date = mapped_column(String)
    
Base.metadata.create_all(engine)
Session = sessionmaker(engine)
session = Session()

We now want to create a function to run all of the collected api links to retreive data from the api and place it into a dataframe. 

In [12]:
def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","x-requested-with":"XMLHttpRequest"} # add x-requested-with headers here to make sure the output is json format

    response = requests.get(url, headers=headers)
    data = response.json()

    if data[str(app_id)]['success']:
        game_data = data[str(app_id)]['data']

        return game_data

In [13]:
for app_id in app_ids:
    game_data = get_game_details(app_id)
    if 'release_date' in game_data:
        date = game_data['release_date']['date']
    else:
        date = 'coming soon'
    if 'price_overview' in game_data:
            price = game_data['price_overview']['final_formatted']
    else:
        price = 'Free' if game_data['is_free'] else 'price not available'
    steam_games = steamGames(
    gameid = game_data['steam_appid'],
    name = game_data['name'],
    description = game_data['short_description'], 
    price = price, 
    date = date) 
    session.add(steam_games)


try: 
     session.commit()
except: 
     session.rollback()
    

In [15]:
from sqlalchemy import select

stmt = select(steamGames).where(steamGames.gameid == "570")
result = session.execute(stmt)

for steamGames_obj in result.scalars(): 
    print(f"{steamGames_obj.gameid} {steamGames_obj.name} {steamGames_obj.price}")

570 Dota 2 Free
